In [1]:
import spacy
from spacy.tokenizer import Tokenizer

import data_preparation as mdp
from mercari_config import MercariConfig

In [2]:
nlp = spacy.load('en')
tokenizer = Tokenizer(nlp.vocab)

In [3]:
word2index = mdp.load_word2index(
    file_name=MercariConfig.WORD_2_INDEX_4_ITEM_DESC_FILE, 
    max_words_from_index=MercariConfig.MAX_WORDS_FROM_INDEX_4_ITEM_DESC)

train_data = mdp.load_data(MercariConfig.TRAINING_SET_PREP_FILE, head=None)

In [4]:
%%time

max_words_item_desc = MercariConfig.MAX_WORDS_IN_ITEM_DESC

for i in range(max_words_item_desc + 1):
    train_data['id' + str(i)] = 0

data_len = len(train_data)

progress = 0

row_iterator = train_data.iterrows()

for index, _ in row_iterator:
    desc = train_data.at[index, 'item_description']
    desc_doc = tokenizer(desc)
    seq_i = 1

    train_data.at[index, 'id0'] = MercariConfig.START_I # <START>

    for token in desc_doc:
        if seq_i <= max_words_item_desc:
            if token.text in word2index.index:
                train_data.at[index, 'id' + str(seq_i)] = word2index.at[token.text, 'word_id']
            else:
                train_data.at[index, 'id' + str(seq_i)] = MercariConfig.OOV_I # <OOV>

            seq_i += 1
        else:
            break

    progress += 1

    if not progress % 10000:
        print("Progress: %3.2f" % (progress * 100.0 / data_len))

Progress: 6.75
Progress: 13.49
Progress: 20.24
Progress: 26.98
Progress: 33.73
Progress: 40.47
Progress: 47.22
Progress: 53.96
Progress: 60.71
Progress: 67.45
Progress: 74.20
Progress: 80.94
Progress: 87.69
Progress: 94.43
CPU times: user 2min 18s, sys: 4.12 s, total: 2min 22s
Wall time: 2min 14s


In [5]:
mdp.save_data(train_data, MercariConfig.TRAINING_SET_PREP_FILE)